In [1]:
import os
import matplotlib.pyplot as plt
if os.name == "posix":
    plt.rc("font", family="AppleGothic")
else:
    plt.rc("font", family="Malgun Gothic")

seed = 1234

In [2]:
import pandas as pd
import numpy as np

import plotly.express as px
import seaborn as sns

import konlpy
from konlpy.tag import Okt, Hannanum
from wordcloud import WordCloud

import re

## 0. 데이터 불러오기

In [3]:
df = pd.read_csv('comments_labeled.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19500 entries, 0 to 19499
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   date      19500 non-null  object
 1   comment   19500 non-null  object
 2   recomm    19500 non-null  int64 
 3   unrecomm  19500 non-null  int64 
 4   labeled   19500 non-null  int64 
dtypes: int64(3), object(2)
memory usage: 761.8+ KB


## 1. 영화 외 댓글 ('labeled' 값이 0) 추출

In [4]:
non_movie_df = df[df['labeled'] == 0]
len(non_movie_df)

10605

In [5]:
text = ' '.join(li for li in non_movie_df['comment'].astype(str))

In [8]:
text = re.sub('[^가-힣]', ' ', text) # 찾을패턴, 변경할 값, 원하는 텍스트 -> 이 방식으로 replace

In [11]:
okt = Okt()
morphemes = okt.morphs(text)

JVMNotFoundException: No JVM shared library file (jvm.dll) found. Try setting up the JAVA_HOME environment variable properly.